In [1]:
from sophius.templates import Conv2dTmpl
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [3]:
from sophius.templates import *
from sophius.encode import Encoder
from sophius.modelgen import ConvModelGenerator

# Example usage
templates = [
    LinearTmpl(),
    BatchNorm2dTmpl(),
    ReLUTmpl(),
    LeakyReLUTmpl(),
    PReLUTmpl(),
    DropoutTmpl(),
    Dropout2dTmpl(),
    FlattenTmpl(),
    Conv2dTmpl(),
    MaxPool2dTmpl(),
    AvgPool2dTmpl(),
    GlobalAvgPool2dTmpl(),
]

encoder = Encoder()

for t in templates:
    encoded_layer = encoder.encode_template(t)    
    print(t.type, t.get_learnable_params())
    print(encoded_layer)
    print()

LinearTmpl {'out_features': 256}
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

BatchNorm2dTmpl {}
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

ReLUTmpl {}
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

LeakyReLUTmpl {'negative_slope': 0.1}
[0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

PReLUTmpl {'all_channels': True}
[0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

DropoutTmpl {'p': 0.75}
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]

Dropout2dTmpl {'p': 0.75}
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]

FlattenTmpl {}
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Conv2dTmpl {'out_channels': 32, 'kernel_size': (3, 3), 'stride': (1, 1), 'padding': True}
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0]

MaxPool2dTmpl {'kernel_size': (2, 2), 'stride': (2, 2), 'padding': False}
[0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 

In [5]:
t = Conv2dTmpl(out_channels=32, kernel_size=(3, 3), stride=(1, 1))
print(encoder.encode_template(t))

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0]


In [11]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=1, lin_num=1)
model_tmpl = model_gen.generate_model_tmpl()

print(model_tmpl)

for t in model_tmpl.templates:
    print(t.type, t.get_learnable_params())

print()

for t in model_tmpl.templates[:-1]:
    print(encoder.encode_template(t))

Conv2d       (8, 10, 10)    (5, 5)   (3, 3)  
PReLU        (8, 10, 10)   
Flatten      800           
Linear       10            

Conv2dTmpl {'out_channels': 8, 'kernel_size': (5, 5), 'stride': (3, 3), 'padding': False}
PReLUTmpl {'all_channels': False}
FlattenTmpl {}
LinearTmpl {'out_features': 10}

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [6]:
for i in range(1000):
    m = model_gen.generate_model_tmpl()
    encoder.model2vec(m)

In [7]:
model_tmpl = model_gen.generate_model_tmpl()
vec = encoder.model2vec(model_tmpl)
print(vec)

[[0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
